In [1]:
from sklearn.model_selection import KFold
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from tensorflow import keras
import tensorflow as tf

In [2]:
dataset = pd.read_csv("../../Database/dataBaseWithNER.csv")
dataset = dataset.drop(columns=["Unnamed: 0"])
dataset = dataset.dropna()
targets = np.array(dataset["target"].array)

dataset

,email,target
0,start date hourahead timee cardinall hou...,0
1,service long desk price structure deal quote ...,0
2,start date cardinall hourahead timee card...,0
3,start date hourahead timee cardinall anc...,0
4,cardinall deliverable revenue management marke...,0
...,...,...
33340,bio matrix scientific group symbo bmxg p...,1
33341,cardinall step away hot naked webcam girl liv...,1
33342,need pill increase performance click seroius ...,1
33343,datee final nom inlet hpl eastrans car...,0


In [3]:
emailsText = []
for email in dataset["email"]:
    emailsText.append(email)

del dataset

In [4]:
print(len(emailsText))

33341


In [5]:
vectorizer = CountVectorizer(max_features=2100)
X = vectorizer.fit_transform(emailsText)

bag = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())

del emailsText
del X

bag

E:\DevPack\anaconda3\envs\data_science\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,aa,ability,able,absolutely,abuse,accept,acceptance,accepted,access,according,...,xanax,xl,xp,yahoo,year,yes,yield,yo,young,zone
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33336,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
33337,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
33338,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33339,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
bag = np.array(bag)
bag

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [7]:
#X_treino, X_teste, y_treino, y_teste = train_test_split(bag.values,target,test_size=0.2)

In [8]:
foldsAccuracy = []
foldLosses = []

In [9]:
kfold = KFold(n_splits=4, shuffle=True)

In [ ]:
foldCount = 1
for train, test in kfold.split(bag, targets):
    model = keras.models.Sequential([
        ########## MLP
        keras.layers.Flatten(input_shape=(bag.shape[1],)),
        #keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(1000, activation="relu"),
        keras.layers.Dense(300, activation="relu"),
        keras.layers.Dense(100, activation="relu"),

        keras.layers.Dense(len(set(targets)), activation="softmax")
    ])

    model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=['accuracy'])

    print('****************************************************')
    print(f'Iniciando treinamento da fold: {foldCount}.')

    callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, min_delta=1e-4,mode='min'), tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, restore_best_weights=True)]

    history = model.fit(bag[train], targets[train], epochs=200, callbacks=callbacks, validation_split=0.05)

    scores = model.evaluate(bag[test], targets[test], verbose=0)
    print(f'Score fold {foldCount}: {model.metrics_names[0]} de {scores[0]}; {model.metrics_names[1]} de {scores[1]*100}%')

    foldsAccuracy.append(scores[1] * 100)
    foldLosses.append(scores[0])

    foldCount = foldCount + 1

****************************************************
Iniciando treinamento da fold: 1.
Epoch 1/200
743/743 [==============================] - 4s 3ms/step - loss: 0.3499 - accuracy: 0.8889 - val_loss: 0.1727 - val_accuracy: 0.9440 - lr: 0.0100
Epoch 2/200
743/743 [==============================] - 2s 2ms/step - loss: 0.2207 - accuracy: 0.9410 - val_loss: 0.1448 - val_accuracy: 0.9528 - lr: 0.0100
Epoch 3/200
743/743 [==============================] - 2s 2ms/step - loss: 0.1933 - accuracy: 0.9501 - val_loss: 0.1184 - val_accuracy: 0.9600 - lr: 0.0100
Epoch 4/200
743/743 [==============================] - 2s 2ms/step - loss: 0.1385 - accuracy: 0.9627 - val_loss: 0.0856 - val_accuracy: 0.9712 - lr: 0.0100
Epoch 5/200
743/743 [==============================] - 2s 2ms/step - loss: 0.1287 - accuracy: 0.9648 - val_loss: 0.1275 - val_accuracy: 0.9576 - lr: 0.0100
Epoch 6/200
743/743 [==============================] - 2s 2ms/step - loss: 0.0776 - accuracy: 0.9768 - val_loss: 0.0639 - val_accurac

In [ ]:
print('****************************************************')
print('Score de cada fold:')
for i in range(0, len(foldsAccuracy)):
    print('****************************************************')
    print(f'--> Fold {i+1}: Loss: {foldLosses[i]} ; Accuracy: {foldsAccuracy[i]}%')

print('****************************************************')
print('Média de accuracy das folds:')
print(f'--> Accuracy: {np.mean(foldsAccuracy)} (+- {np.std(foldsAccuracy)})')
print(f'--> Loss: {np.mean(foldLosses)}')
print('****************************************************')